## Preprocess SST

In [ ]:
SST_DIR = '/projets/EMC2/data/Observations/SST/'

ersst = xr.open_dataset(f'{SST_DIR}/ERSSTv5.nc').sst
cobe2 = xr.open_dataset(f'{SST_DIR}/COBE2.nc').sst
hadisst = xr.open_dataset(f'{SST_DIR}/HadISST_sst.nc').sst

hadisst = hadisst.where(hadisst>-100)
hadisst = hadisst.where(hadisst.count('time')==hadisst.count('time').max())

grid_out = xr.Dataset({'lat': (['lat'], np.arange(-89, 90, 2.)),
                          'lon': (['lon'], np.arange(1, 360, 2.))})

regridder_hadisst = xe.Regridder(hadisst, grid_out, 'conservative_normed', periodic=True)
regridder_cobe2 = xe.Regridder(cobe2, grid_out, 'conservative_normed', periodic=True)
regrdder_ersst = xe.Regridder(ersst, grid_out, 'conservative_normed', periodic=True) 

hadisst = regridder_hadisst(hadisst)
cobe2 = regridder_cobe2(cobe2)
ersst = regrdder_ersst(ersst)

ersst = ersst.assign_coords(time=pd.date_range(f"{ersst.time.dt.year.values[0]}-{ersst.time.dt.month.values[0]}",freq='MS', periods=ersst.time.size))
cobe2 = cobe2.assign_coords(time=pd.date_range(f"{cobe2.time.dt.year.values[0]}-{cobe2.time.dt.month.values[0]}",freq='MS', periods=cobe2.time.size))
hadisst = hadisst.assign_coords(time=pd.date_range(f"{hadisst.time.dt.year.values[0]}-{hadisst.time.dt.month.values[0]}",freq='MS', periods=hadisst.time.size))

ds_tos = xr.concat([ersst.assign_coords(product='ERSSTv5'),
                    cobe2.assign_coords(product='COBE2'),
                    hadisst.assign_coords(product='HadISST'),],dim='product')

ds_tos = ds_tos.squeeze().drop_vars('lev')
ds_tos = ds_tos.sortby('lat', ascending=False)
ds_tos = ds_tos.where(ds_tos.count('product')==ds_tos.count('product').max())

/home/rguillaume/virtual_envs/phd/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


## Preprocess Green's Functions

In [ ]:
GF_names = dict(cam4='CAM4',cam5='CAM5', canesm5='CanESM5', echam6='ECHAM6', gfdlam4='GFDL-AM4',hadcm3='HadCM3')

GF_DIR = '/projets/EMC2/data/Models/GFMIP/preliminary_spatial_feedbacks-main/data/'
list_gfs = [k.split('.')[0] for k in os.listdir(GF_DIR)]
all_gfs = []
for gf in list_gfs:
    ds = xr.open_dataset(f"{GF_DIR}/{gf}.nc").rename(longitude='lon', latitude='lat')
    if gf == 'hadcm3':
        ds = ds.assign_coords(latitude_bnds=ds.latitude_bnds.where(np.abs(ds.latitude_bnds)!=90, -ds.latitude_bnds))
    if gf=='cam4':
        da = ds.spatial_feedbacks_from_warming
    else:
        da = ds.spatial_feedbacks_from_both
    ds['mask'] = (da!=0).astype(int)
    regridder_gf = xe.Regridder(ds, grid_out, 'conservative_normed', periodic=True)
    GF_regridded = regridder_gf(da).assign_coords(GF_source=gf)
    all_gfs.append(GF_regridded)
GF = xr.concat(all_gfs, dim='GF_source')

GF = GF.where(GF!=0)
# GF = GF.where(GF.count('GF_source')==GF.count('GF_source').max(),0)
GF = xr.concat([GF, GF.mean('GF_source').assign_coords(GF_source='mean')], dim='GF_source')

cell_area = xr.zeros_like(GF.isel(GF_source=0))+np.cos(np.deg2rad(GF.lat))
cell_ratio = cell_area/cell_area.sum(['lon','lat'])
GF = (GF*cell_ratio).fillna(0)